<a href="https://colab.research.google.com/github/IvanCampos11/DS-Unit-4-Sprint-2-Neural-Networks/blob/main/module4-Deploy/LS_DS_434_Deploy_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [1]:
# Your Code Starts Here
# {'num_units': 32, 'learning_rate': 0.001, 'optimizer': 'adam'}
%load_ext tensorboard

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [3]:
def load_quickdraw10(path):
  data = np.load(path)
  X = data['arr_0']
  y = data['arr_1']
  X, y = shuffle(X, y)


  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

  return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = load_quickdraw10('quickdraw10.npz')

In [5]:
# X_train = X_train / 255.0
# X_test = X_test / 255.0

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU
import tensorflow as tf
import os

logdir = os.path.join("logs", "EarlyStopping-Loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential([
    Flatten(),
    Dense(512),
    ReLU(negative_slope=.01),
    Dense(512),
    ReLU(negative_slope=.01),
    Dense(512),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
   1/2500 [..............................] - ETA: 0s - loss: 82.7688 - accuracy: 0.1562WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2500/2500 [==============================] - 9s 4ms/step - loss: 1.3554 - accuracy: 0.7563 - val_loss: 0.6166 - val_accuracy: 0.8130
Epoch 2/99
2491/2500 [============================>.] - ETA: 0s - loss: 0.5580 - accuracy: 0.8308

In [ ]:
from tensorflow.keras import regularizers

logdir = os.path.join("logs", "EarlyStopping+L2_WeightDecay")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential([
    Flatten(),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(512, kernel_regularizer=regularizers.l2(0.01)),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

In [ ]:
%tensorboard --logdir logs

In [ ]:
from tensorflow.keras.constraints import MaxNorm

wc = MaxNorm(max_value=2)

logdir = os.path.join("logs", "EarlyStopping+WeightConstraint")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential([
    Flatten(),  
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

In [ ]:
from tensorflow.keras.layers import Dropout

from tensorflow.keras.constraints import MaxNorm

wc = MaxNorm(max_value=2)

logdir = os.path.join("logs", "EarlyStopping+WeightConstraint+Dropout")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential([
    Flatten(),
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(512, kernel_constraint=wc),
    ReLU(negative_slope=.01),
    Dropout(.2),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

In [ ]:
%tensorboard --logdir logs

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [ ]:
# Your Code Starts Here
import tensorflow as tf

cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h5",
                                            verbose=1, 
                                            save_weights_only=True)

def create_model():
  model = tf.keras.Sequential([
      Flatten(),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(128),
      ReLU(negative_slope=.01),
      Dense(10, activation='softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
                metrics=['accuracy'])

  return model

model = create_model()

model.fit(X_train, y_train, epochs=2, 
          validation_data=(X_test,y_test),
          verbose=2,
          callbacks=[cpoint])

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 